Final Project Check-in 2018-11-16
------

Group Name
-----
Data Explorers

Student Names
----

1. Adam Reevesman
2. Gokul Krishna
3. Hai Le
4. Maximillian Alfaro 
5. Prakhar Agrawal:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from pathlib import Path
import re
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Load Data
-----

In [2]:
path = Path('Data/')
df = pd.read_feather('Data/May2015_subset.fthr')

/anaconda3/envs/ml/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [39]:
df.tail(3).T

,6195169,6195170,6195171
created_utc,1433116781,1433116784,1433116785
ups,170,9,37
link_id,t3_37zl5z,t3_380j06,t3_37z0mw
name,t1_crrbedn,t1_crrbefm,t1_crrbeft
subreddit,worldnews,worldnews,worldnews
id,crrbedn,crrbefm,crrbeft
score,170,9,37
body,Because it's not typically done to babies It's...,&gt Nothing they are doing is illegal under in...,Apart from the risk of inhalation pneumonia
distinguished,NaN,NaN,NaN
controversiality,0,0,0


Preprocessing and Feature Engineering
-----

First is to exclude columns that are not helpful.

In [4]:
excluded_var = ['subreddit_id', 'score_hidden', 'author_flair_css_class', 
                'author_flair_text', 'removal_reason', 'gilded', 'downs', 'archived', 
                'author', 'retrieved_on', 'edited']
df = df.drop(excluded_var, axis=1)

In [5]:
df['subreddit'] = df.subreddit.astype('category')
# fill na in `distinguised` with 0
df['distinguished'].fillna('None', inplace=True)
df['distinguished'] = df.distinguished.astype('category')
df['link_id'] = df.link_id.astype('category')

And make sure the data is in the format we want, eg. for time data to be in datetime format.

In [6]:
# Convert into date-time type
df['time'] = pd.to_datetime(df.created_utc, unit='s')

Then we create features based on the raw data we have.

In [7]:
# Compute the time lapse of each comment compared with the first one
df_first_comment = df[['link_id', 'time']].groupby(by=['link_id']).min()
df_first_comment_dict = df_first_comment.to_dict('index')

def time_zero(x):
    return df_first_comment_dict[x]['time']

df['time_lapse'] = df.time - df.link_id.apply(time_zero)

df['time_lapse'] = df.time_lapse.apply(lambda x: x.seconds)  # convert into seconds

In [8]:
# Compute the depth of the comment in the thread
def pd_to_dict(df, index='', column=''):
    dft = df.copy(deep=True)
    dft.index = dft[index]
    dft = dft.drop([index], axis=1)
    res = dft.to_dict()[column]
    del dft
    return res

comment_to_parent_dict = pd_to_dict(df[['name', 'parent_id']],
                                    index='name', column='parent_id')

def get_depth(x):
    counter = 0
    temp = x
    while True:
        if temp[:3] == 't3_':
            return counter
        try:
            temp = comment_to_parent_dict[temp]
        except KeyError:
            return np.nan
        counter += 1
        
df['depth'] = df['name'].apply(get_depth)

A fraction of records doesn't have a proper `parent_id` which results in nulls in our derived feature `depth`. So we will remove those rows.

In [9]:
df = df[df.depth.isna() == False]

Now we can also get rid of unuseful records, ie. those whose comment text was deleted.

In [24]:
df = df[~(df.body == '[deleted]')]

In [26]:
# Comment types: response to other comments or to links
df['respond_to_comment']=np.where(df['parent_id'].str.startswith('t1'), 1,0)
df['respond_to_link']=np.where(df['parent_id'].str.startswith('t3'),1,0)

In [53]:
# Subjectivity of the comments
df['subjectivity'] = df['body'].str.count('i ')

In [58]:
# word count 
df['body'] = df['body'].str.replace('[.,?!;:]', '')  # remove punctuations
df['word_count'] = df['body'].str.split().str.len()

# remove comments that have  more than 300 words (outliers)
df = df[df['word_count'] < 300]

In [59]:
# word count categorical variable
bins=[-1, 25, 50, 100, 300]
df['word_count_cat'] = pd.cut(df['word_count'], 
                                  bins, labels=['short', 'medium', 'long', 'very long'])

In [32]:
df['linked_sr'] = df.body.apply(lambda x: re.findall(r"/r/([^\s/]+)", x))
df['no_of_linked_sr'] = df['linked_sr'].apply(lambda x: len(x))

In [40]:
df['weekday'] = df['time'].dt.dayofweek
df['weekday'] = df.weekday.astype('category')

In [60]:
df.head(3).T

,1,11,14
created_utc,1430438411,1430438501,1430438525
ups,12,1,2
link_id,t3_34gfze,t3_34gma5,t3_34gma5
name,t1_cqug99n,t1_cqugb3u,t1_cqugbkt
subreddit,anime,anime,anime
id,cqug99n,cqugb3u,cqugbkt
score,12,1,2
body,I could participate in this but by that time y...,There are shows that I'm sure are brilliant bu...,Good writing Good directing Good sound design
distinguished,NaN,NaN,NaN
controversiality,0,0,0


Fit scikit-learn model
----

In [34]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [ ]:
binary_variables = []
multi_variables = ['subreddit', 'distinguished',  ]

In [61]:
df.columns

Index(['created_utc', 'ups', 'link_id', 'name', 'subreddit', 'id', 'score',
       'body', 'distinguished', 'controversiality', 'parent_id', 'time',
       'time_lapse', 'depth', 'respond_to_comment', 'respond_to_link',
       'subjectivity', 'word_count', 'linked_sr', 'no_of_linked_sr', 'weekday',
       'word_count_cat'],
      dtype='object')

In [ ]:
y = comments.ups.values
x = np.concatenate((subreddit_lb.fit_transform(comments.subreddit),
                    meta_lb.fit_transform(comments.meta),
                    # linked_sr_lb.fit_transform(comments.linked_sr),
                    linked_urls_lb.fit_transform(comments.a_linked_urls),
                    comments.loc[:, col_numeric].values), axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                                    random_state=3)

In [ ]:
lm_model = LinearRegression()
lm_model.fit(X_train, y_train)
y_predicted = lm_model.predict(X_test)

Evaluation Metric
----

In [ ]:
mse = mean_squared_error(y_test, y_predicted)
print(f"{mse:.4}")

In [ ]:
r2 = r2_score(y_test, y_predicted)
print(f"{r2:.4}")